# Individuals
- The base unit of an evolutionary algorithm (EA) is the individual.
- An individual represents a single solution to the problem we want to solve.
- Classic EAs often have binary individuals, where each gene is represented by a bit (0/1 value).

In [15]:
import numpy as np

In [16]:
class Individual:
    def __init__(self, n: int):
        self.genes = np.random.randint(0, 2, (n,))
        self.fitness = -np.inf
    def __str__(self):
        return f'(Ind: {self.genes}, {self.fitness})'
    def __repr__(self):
        return str(self)


- Individual has binary genes and an integer fitness.
- A newly-initialized individual often has random genes.

In [17]:
ind = Individual(10)
ind


(Ind: [1 1 0 0 1 0 1 0 0 0], -inf)

# Objectives
- The objective function gives a value to our individual.
- The objective function can be designed so that it can evaluate the individual.
- The objective function value of an individual is often called the fitness value of that individual.
- The objective function does not need to be differentiable or continuous.

In [18]:
def onemax(i: Individual):
    return np.sum(i.genes)


The OneMax function simply adds all the bits of the genotype of the individual.

In [19]:
ind.fitness = onemax(ind)
ind

(Ind: [1 1 0 0 1 0 1 0 0 0], 4)

The optimal solution for OneMax is the individual that has the genotype of all 1s.


To be more general, we often define an evaluate function, which evaluate the fitness value of an individual.



In [20]:
def evaluate(ind: Individual, objective):
    ind.fitness = objective(ind)

evaluate(ind, onemax)
ind


(Ind: [1 1 0 0 1 0 1 0 0 0], 4)

- Each time we evaluate the fitness value of an individual, we spend one evaluation function call.
- To be abstract from the exact runtime on specific hardware, we often use the number of evaluations as the computing cost/budget for EAs.


# (1+1) Evolutionary Algorithm ~~~ (1+1)-EA

In [21]:
n = 20

In [22]:
parent = Individual(n)
evaluate(parent, onemax)
parent


(Ind: [0 1 0 1 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 0], 12)

In [23]:
def mutate(ind: Individual, mutation_rate=1.0/len(ind.genes)):
    new_genes = np.copy(ind.genes)
    for i in range(len(new_genes)):
        if np.random.rand() < mutation_rate:
            new_genes[i] = not ind.genes[i]
    child = Individual(len(ind.genes))
    child.genes = new_genes
    return child


- The mutate function create a child/offspring individual from a parent individual with the mutation probability of p = 1/n 
- We scan through each gene in an individual.
- For each gene, we generate a random number r∈(0,1).
- If r<p, we flip the gene value 0↔1.


In [24]:
child = mutate(parent)
print("Parent :", parent)
print("Child  :", child)
print("Genes :", parent.genes == child.genes)


Parent : (Ind: [0 1 0 1 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 0], 12)
Child  : (Ind: [0 1 0 1 1 1 0 1 0 1 1 0 0 1 1 0 1 1 1 0], -inf)
Genes : [ True  True  True  True  True  True  True  True  True  True  True False
  True False  True  True  True  True  True  True]


- Running this mutation function a few times.
- Each time, only one or two genes change because the mutation rate is set to p = 1/n  

In [25]:
child = mutate(parent)
print("Parent :", parent)
print("Child  :", child)
print("Genes :", parent.genes == child.genes)


Parent : (Ind: [0 1 0 1 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 0], 12)
Child  : (Ind: [0 1 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 1 1 0], -inf)
Genes : [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True False  True  True  True  True]


In [26]:
child = mutate(parent)
print("Parent :", parent)
print("Child  :", child)
print("Genes :", parent.genes == child.genes)


Parent : (Ind: [0 1 0 1 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 0], 12)
Child  : (Ind: [0 1 0 1 1 1 0 1 1 1 1 1 0 0 1 0 1 1 1 0], -inf)
Genes : [ True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True  True  True]


Each time, we replace the parent individual x by the child individual x′ if the fitness of the offspring is better, i.e., f(x′)≥f(x).


In [27]:
evaluate(child, onemax)
print(parent.fitness)
print(child.fitness)
if child.fitness >= parent.fitness:
    parent = child
parent.fitness


12
13


13

- We simply run this mutation step over and over until we reach the solution we want or the certain termination criterion is met (e.g., the maximum number of evaluations are all spent).
- Let's just run for a few iterations.


In [34]:
parent = Individual(n)
for i in range(20):
    child = mutate(parent)
    evaluate(child, onemax)
    if child.fitness >= parent.fitness:
        parent = child
    # print(i, " ", parent.fitness)


print(parent.fitness, " / ", n)

15  /  20


- Let's code the (1+1)-EA.


In [45]:
from tqdm import tqdm

def one_plus_one(ind_length: int, num_generations: int, objective):
    fits = np.zeros(num_generations)
    parent = Individual(ind_length)
    evaluate(parent, objective)
    
    for i in tqdm(range(len(fits))):
        child = mutate(parent)
        evaluate(child, objective)

        if child.fitness >= parent.fitness:
            parent = child
            
        fits[i] = parent.fitness
    return fits

- Since EAs are stochastic algorithm, it's difficult to guarantee their exact computational complexity.
- A common metric is the expected number of generations to each the optimal solution.
- The worst-case for a binary (1+1)-EA to converge is O(n^n), but we often don't need to run that long to see convergence.
- For OneMax, the expected runtime for (1+1)-EA has been proven to be O(nlogn) when the mutation rate is p = 1/n
 

In [46]:
n = 50
fits = one_plus_one(n, int(np.round(n * np.log(n))), onemax)
print(fits[-1])


TypeError: 'module' object is not callable